# Simple Q- Learning
### Finding Ten Discrete Meals To Fit Calorie Requirements

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Reading in nutrition and reqs

In [2]:
df = pd.read_csv('C:/Users/J/Desktop/Businesses/Meal_Maker/Scraped_Data/combined_nutrition_small/nutrition_sm_processed_ss.csv',dtype = {'food_key':int,'ingredients_list':object}, encoding='ISO-8859-1')

In [3]:
df = df.fillna(0)

In [4]:
num_reqs_df = pd.read_csv("C:/Users/J/Fitness/Meal Plans/num_reqs_df.csv")

In [26]:
num_reqs_df

,calories,protein_g,fat_g,saturated_fat_g,carb_g,fiber_g,sugar_g,sodium_mg,cholesterol_mg,calcium_mg,iron_mg,vit_a_mcg,vit_c_mg
0,2226.178233,222.617823,86.573598,12.367657,139.13614,31.166495,27.827228,2400,300,1000,18,5000,60


### Model Set Up

In [18]:
tf.reset_default_graph()

States

In [19]:
# float vector 1 - 10 for each 'state'
inputs = tf.placeholder(shape=[1,10],dtype=tf.float32)

Actions

In [20]:
# randomized weights for each actions
W = tf.Variable(tf.random_uniform([10, len(df)],0,0.01))

Matrix multiply

In [21]:
# Matrix multily the inputs (states) times the weights (actions)
Qout = tf.matmul(inputs, W)
# Choose index of the largest value in the output accross the 1st axis (row) - choosing the largest expected 'reward'
predict = tf.argmax(Qout, 1)

In [22]:
# inserts a placeholder for a tensor that will always be fed into the algorithm. Is the shape of one possible action
# Seem to be trained one state at a time
nextQ = tf.placeholder(shape=[1,len(df)], dtype=tf.float32)

In [23]:
# Loss function - does this include all previous states?
loss = tf.reduce_sum(tf.square(nextQ-Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate = 0.1)

In [24]:
updateModel = trainer.minimize(loss)

### Running Model

In [ ]:
init = tf.initialize_all_variables()

# Setting learning parameters
y=.99 
e =0.1
num_episodes = 2000
# Lists to capture total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        # Reset environment and get first observation - need to tailor to meal plan class
        s = env.reset()
        # Total expected reward value at state
        rAll = 0
        # because this is feed-forward, can set d to be going over by 75 calories. Can't work backwards, so bust
        d = False 
        
        # Q Network Iteration
        j = 0
        while j < 99:
            j+=1
            # Choose an action greedily from Q-Network. Includes e chance of random action
            a, allQ = sess.run([predict, Qout], feed_dict={inputs:np.identity(10)[s:s+1]})
            # Reset a[0] according to randomness 
            if np.random.rand(1) < e:
                a[0]= env.action_space.sample()
            # Get New State And Reward From Environment
            # Short term rewards should be not going over the limit 
            